In [1]:
import json
import pandas as pd 

In [2]:
import chart_studio
import chart_studio.plotly as cs_py


username = 'piotrsobecki' # your username
api_key = 'axrDPVVhymNzGrTX3Tyj' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)



In [3]:
patents_df = pd.read_excel('out/inst_patents.xlsx')


In [4]:
nace = pd.read_excel('concordance/nace.xlsx')


In [5]:
nace_code_labels  = nace[(nace['Code'].str.len() <=2) ].to_dict(orient='records')

code_label_mapping = {d['Code']:d['Code'] + ' - ' +d['Label'] for d in nace_code_labels}

In [6]:
nace_ipc_df = pd.read_excel('concordance/nace_ipc.xlsx', index_col='NACE')

# Nace is a 2 digit code so append 0 as the prefix where necessary
nace_ipc_df.index = nace_ipc_df.index.map(lambda x: '0'+str(x) if len(str(x)) == 1 else str(x))

In [7]:
import numpy as np
# Visualize NACE-IPC concordance table using plotly heatmap
def visualize_nace_ipc(nace_ipc_df):
    import plotly.graph_objects as go
    
    
    fig = go.Figure(data=go.Heatmap(
                   z=nace_ipc_df.values,
                   x=nace_ipc_df.columns,
                   y=[ code_label_mapping[str(i)] for i in nace_ipc_df.index],
                   colorscale='Viridis'))
    fig['layout']['yaxis']['autorange'] = "reversed"
    fig.update_layout(
        #height=1000, width=2000, 
                      title_text='NACE-IPC concordance table')
    return fig
    
fig = visualize_nace_ipc(np.power(nace_ipc_df, 0.25))
cs_py.plot(fig, filename = 'nace_ipc_heatmap', auto_open=False)
fig

In [8]:

pl_df = nace_ipc_df.reset_index().melt(id_vars='NACE',  value_name='value').rename(columns={'variable':'IPC'})


In [9]:
pl_df

,NACE,IPC,value
0,01,A01,0.009865
1,02,A01,0.000334
2,03,A01,0.000000
3,05,A01,0.000000
4,06,A01,0.000000
...,...,...,...
10907,95,H05,0.000393
10908,96,H05,0.001081
10909,97,H05,0.000000
10910,98,H05,0.000000


In [10]:
labels = pl_df['NACE'].unique().tolist() +  pl_df['IPC'].unique().tolist()

In [11]:
pl_df['NACE_i'] = pl_df['NACE'].apply(lambda x: labels.index(x))
pl_df['IPC_i'] = pl_df['IPC'].apply(lambda x: labels.index(x))


In [12]:
pl_data = pl_df.rename(columns={'IPC_i':'source', 'NACE_i':'target', 'value':'weight'})

# https://ec.europa.eu/eurostat/ramon/nomenclatures/index.cfm?TargetUrl=LST_NOM_DTL_LINEAR&IntCurrentPage=1&StrNom=CL_IPC&StrLanguageCode=EN

In [13]:
pl_data

,NACE,IPC,weight,target,source
0,01,A01,0.009865,0,88
1,02,A01,0.000334,1,88
2,03,A01,0.000000,2,88
3,05,A01,0.000000,3,88
4,06,A01,0.000000,4,88
...,...,...,...,...,...
10907,95,H05,0.000393,83,211
10908,96,H05,0.001081,84,211
10909,97,H05,0.000000,85,211
10910,98,H05,0.000000,86,211


In [14]:
import plotly.graph_objects as go

# Plot nace_ipc mapping data using Sankey chart visualizing mapping IPC to NACE codes 
def plot_sankey(df, labels, title):
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = labels
        ),
        link = dict(
          source = df['source'], # indices correspond to labels, eg A1, A2, A2, B1, ...
          target = df['target'],
          value = df['weight']
        ))])

    fig.update_layout(title_text=title, font_size=10, height=1000)
    return fig
    
fig = plot_sankey(pl_data, labels, 'IPC to NACE mapping')
fig.show()

In [15]:
patents_df['ipc_3'] = patents_df['ipc_classification'].str.slice(0,3)


In [16]:
nace_ipc_df.head(1)

,A01,A21,A22,A23,A24,A41,A42,A43,A44,A45,...,G10,G11,G12,G16,G21,H01,H02,H03,H04,H05
NACE,,,,,,,,,,,,,,,,,,,,,
01,0.009865,0.0,0.013158,0.000314,0.0,0.013841,0.0,0.001618,0.0,0.004299,...,0.001308,0.0,0.0,0.0,0.001304,0.002258,0.001889,0.000217,0.002533,0.002063


In [17]:
import json
from json import JSONDecodeError
import ast
import numpy as np

dummy_row = nace_ipc_df.T.head(0)
pds = []

# Get secondary ipc classification based on list of nace codes - if nan return None, if str return list
def get_ipc(nace_codes):
    if type(nace_codes) == float:
        return []
    if type(nace_codes) == str:
        
        if ',' in nace_codes:
            try:
                return  ast.literal_eval(nace_codes)# [nace_codes]
            except SyntaxError as e:
                pass
        return [nace_codes]
    if type(nace_codes) == list:
        return nace_codes
    return None

def get_ipc_vect(ipc):
    ipc3 = ipc[:3]
    if ipc3 not in nace_ipc_df.columns:
        return None
    return nace_ipc_df[ipc3].values
    

def combine(main_ipc, secondary_ipcs, w1=0.5, w2=0.5):
    main_ipc = get_ipc_vect(main_ipc)
    secondary_ipcs = [get_ipc_vect(ipc) for ipc in secondary_ipcs]
    secondary_ipcs = [v for v in secondary_ipcs if v is not None]
   
    # Filter main_ipc
    if main_ipc is None:
        w1 = 0
        main_ipc = np.zeros(dummy_row.shape[1])
    
    if secondary_ipcs is None or len(secondary_ipcs) == 0:
        w2 = 0 
        secondary_ipcs_v = np.zeros(dummy_row.shape[1])
    else: 
        secondary_ipcs_v = np.sum(secondary_ipcs, axis=0) / len(secondary_ipcs)
    
    
    if (w1+w2) == 0:
        result = np.zeros(dummy_row.shape[1])
    else: 
        result =  (w1 * main_ipc + w2 * secondary_ipcs_v) / (w1 + w2)
    
    #print('Main', np.sum(main_ipc), 'Secondary', np.sum(secondary_ipcs_v), 'Result', np.sum(result))
    return result
    


w_main = 0.5# 0.5
w_secondary = 0.5#0.5

for p_id in range(len(patents_df)):
    ipc_main = get_ipc(patents_df.iloc[p_id]['ipc_classification'])
    if ipc_main is None or len(ipc_main) == 0:
        pds.append(dummy_row.values)
        continue
    
    ipc_sec = get_ipc(patents_df.iloc[p_id]['ipc_classification_secondary'])
    
    pds.append(combine(ipc_main[0], ipc_sec, w_main, w_secondary))


out_df = pd.DataFrame(pds,columns=dummy_row.columns)
out_df.columns=[ '0' * (2-len(str(col))) + str(col) for col in out_df.columns]
out_df.index = patents_df.index

In [18]:
out_df_sectors = pd.DataFrame()

columns2 = []


for col in out_df:
    sector = nace[nace['Code'] == col]['Sector'].values[0]
    columns2.append([sector,col])
    if sector in out_df_sectors.columns:
        out_df_sectors[sector] = out_df_sectors[sector] + out_df[col]
    else:
        out_df_sectors[sector] = out_df[col]

out_df_sectors.index = patents_df.index

In [19]:
# Add secondary level to out_df column names using columns2 list
out_df.columns=pd.MultiIndex.from_tuples(columns2)

#out_df.columns = columns2

In [20]:
out_df

A                             B                                \
             01        02        03        05        06        07        08   
0      0.003045  0.000428  0.000000  0.000587  0.002347  0.000587  0.006527   
1      0.001852  0.000093  0.000000  0.000000  0.005247  0.000031  0.000309   
2      0.001269  0.000111  0.000000  0.000059  0.001071  0.000000  0.000081   
3      0.008689  0.000000  0.000000  0.000000  0.000644  0.000805  0.000000   
4      0.003045  0.000428  0.000000  0.000587  0.002347  0.000587  0.006527   
...         ...       ...       ...       ...       ...       ...       ...   
35614  0.001840  0.000000  0.000016  0.000127  0.010379  0.002906  0.006652   
35615  0.001840  0.000000  0.000016  0.000127  0.010379  0.002906  0.006652   
35616  0.001840  0.000000  0.000016  0.000127  0.010379  0.002906  0.006652   
35617  0.001840  0.000000  0.000016  0.000127  0.010379  0.002906  0.006652   
35618  0.006631  0.000000  0.000000  0.000000  0.004374  0.000820  0.000547   

                        C            ...         R                      \
             09        10        11  ...        90        91        92   
0      0.002887  0.001499  0.000000  ...  0.000214  0.000000  0.000000   
1      0.034751  0.002284  0.000648  ...  0.000031  0.000062  0.000123   
2      0.009139  0.000473  0.000000  ...  0.000273  0.000000  0.000000   
3      0.002896  0.001770  0.000000  ...  0.000000  0.000000  0.000000   
4      0.002887  0.001499  0.000000  ...  0.000214  0.000000  0.000000   
...         ...       ...       ...  ...       ...       ...       ...   
35614  0.007094  0.010456  0.000776  ...  0.000040  0.000072  0.000000   
35615  0.007094  0.010456  0.000776  ...  0.000040  0.000072  0.000000   
35616  0.007094  0.010456  0.000776  ...  0.000040  0.000072  0.000000   
35617  0.007094  0.010456  0.000776  ...  0.000040  0.000072  0.000000   
35618  0.005741  0.001043  0.000000  ...  0.000000  0.000000  0.000000   

                        S                        T         U  
             93        94        95        96   97   98   99  
0      0.000214  0.001229  0.000428  0.001872  0.0  0.0  0.0  
1      0.000154  0.004043  0.000370  0.000679  0.0  0.0  0.0  
2      0.000163  0.000532  0.000222  0.000776  0.0  0.0  0.0  
3      0.000000  0.000322  0.000000  0.000965  0.0  0.0  0.0  
4      0.000214  0.001229  0.000428  0.001872  0.0  0.0  0.0  
...         ...       ...       ...       ...  ...  ...  ...  
35614  0.000000  0.001230  0.000024  0.003058  0.0  0.0  0.0  
35615  0.000000  0.001230  0.000024  0.003058  0.0  0.0  0.0  
35616  0.000000  0.001230  0.000024  0.003058  0.0  0.0  0.0  
35617  0.000000  0.001230  0.000024  0.003058  0.0  0.0  0.0  
35618  0.000000  0.001068  0.000000  0.008145  0.0  0.0  0.0  

[35619 rows x 88 columns]

In [21]:
out_df.to_excel('out/patents_nace.xlsx')
#out_df_sectors.to_excel('out/patents_nace_sectors.xlsx')
